In [1]:
from src.utils import save, load
import numpy as np

/home/steinad/anaconda3/envs/lexx/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
distilbert_topic_vals = load("topic_vals_distilbertbase_sst2").reshape(872, 73)
robertalarge_topic_vals = load("topic_vals_robertalarge_sst2").reshape(872, 73)

In [18]:
diffs = np.sum(np.abs(distilbert_topic_vals - robertalarge_topic_vals), axis=1)
biggest_diff = np.argsort(diffs)
print(diffs[biggest_diff[-5:]])

[1.36153851 1.37051871 1.47261364 1.48859745 1.70616717]


In [25]:
from datasets import load_dataset

sst2 = load_dataset("sst2", split="validation")
x = [sst2[i]['sentence'] for i in range(len(sst2))]

Found cached dataset sst2 (/home/steinad/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


In [22]:
from src.pair_data import LIWCWordData

def get_liwc_topics():
    feature_groups = LIWCWordData(
        "../data/LIWC2015_processed.csv",
        split="train")

    print(feature_groups.groups)
    topic_names = feature_groups.groups
    topics = {name: set() for name in topic_names}
    for word, groups in feature_groups:
        for group in groups:
            topics[group].add(word)
    all_tokens = set().union(*topics.values())
    word2idx = {word: i for i, word in enumerate(all_tokens)}
    topics = np.array([[1.0 if tok in topics[topic_names[i]]
                        else 0.0 for tok in all_tokens] for i in range(len(topic_names))])
    topics = topics / np.sum(topics, axis=1, keepdims=True)
    return topics, topic_names, word2idx

topics, topic_names, word2idx = get_liwc_topics()

['ACHIEVE' 'ADJ' 'ADVERB' 'AFFECT' 'AFFILIATION' 'ANGER' 'ANX' 'ARTICLE'
 'ASSENT' 'AUXVERB' 'BIO' 'BODY' 'CAUSE' 'CERTAIN' 'COGPROC' 'COMPARE'
 'CONJ' 'DEATH' 'DIFFER' 'DISCREP' 'DRIVES' 'FAMILY' 'FEEL' 'FEMALE'
 'FILLER' 'FOCUSFUTURE' 'FOCUSPAST' 'FOCUSPRESENT' 'FRIEND' 'FUNCTION'
 'HEALTH' 'HEAR' 'HOME' 'I' 'INFORMAL' 'INGEST' 'INSIGHT' 'INTERROG'
 'IPRON' 'LEISURE' 'MALE' 'MONEY' 'MOTION' 'NEGATE' 'NEGEMO' 'NETSPEAK'
 'NONFLU' 'NUMBER' 'PERCEPT' 'POSEMO' 'POWER' 'PPRON' 'PREP' 'PRONOUN'
 'QUANT' 'RELATIV' 'RELIG' 'REWARD' 'RISK' 'SAD' 'SEE' 'SEXUAL' 'SHEHE'
 'SOCIAL' 'SPACE' 'SWEAR' 'TENTAT' 'THEY' 'TIME' 'VERB' 'WE' 'WORK' 'YOU']


In [35]:
def sort_shap(shap_values, feature_names):
    sort_idx = np.argsort(shap_values)
    return shap_values[sort_idx], feature_names[sort_idx]

print(x[biggest_diff[-1]])
topic_values, sorted_names = sort_shap(distilbert_topic_vals[biggest_diff[-1]], topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")
print(np.sum(topic_values))

print()
topic_values, sorted_names = sort_shap(robertalarge_topic_vals[biggest_diff[-1]], topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")
print(np.sum(topic_values))

an entertaining , colorful , action-filled crime story with an intimate heart . 
ANGER: -1.8504402262217918e-05
NEGEMO: -5.750461162503181e-06
AFFECT: -3.1536017117971704e-06
ACHIEVE: 0.0
POWER: 0.0
HEAR: 0.0
PREP: 0.0005175859107903193
SOCIAL: 0.0005255784083522601
FUNCTION: 0.0030943426793281544
ARTICLE: 0.5953518952809127
0.5994619938142469

ARTICLE: -1.055920267974337
SOCIAL: -0.0013257281632984923
FUNCTION: -0.0009159737743029677
ACHIEVE: 0.0
POWER: 0.0
FOCUSPAST: 0.0
AFFECT: 0.002035189891602397
NEGEMO: 0.003711083865219365
ANGER: 0.011941892437884887
PREP: 0.031835397615878266
-1.0086384061013536
